In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('spambase.csv',header=None)

In [3]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [4]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [5]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: 57, dtype: int64

In [6]:
def zscore_normalize_features(X):

    mu     = np.mean(X, axis=0)                 
    sigma  = np.std(X, axis=0)
    
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

In [7]:
def sigmoid(z):
   
    g=np.zeros_like(z)
    
    g=1/(1+np.exp(-z))
        
    return g

In [8]:
def compute_cost(X, y, w, b):

    m, n = X.shape

    z_wb = np.dot(X, w) + b
    f_wb = sigmoid(z_wb)

    loss = -y * np.log(f_wb) - (1 - y) * np.log(1 - f_wb)
    cost = np.mean(loss)

    return cost

In [9]:
def compute_gradient(X, y, w, b): 
  
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    z_wb = np.dot(X, w) + b
    f_wb = sigmoid(z_wb)

    dj_db = np.sum(f_wb - y)
    dj_dw = np.dot((f_wb - y), X) / m

    return dj_db, dj_dw

In [18]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    
    m = len(X)
    
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        dj_db, dj_dw = gradient_function(X, y, w_in, b_in)   

        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        if i<100000:     
            cost =  cost_function(X, y, w_in, b_in)
            J_history.append(cost)

        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train_norm,mu,sigma= zscore_normalize_features(X_train)
X_test_norm,mu,sigma= zscore_normalize_features(X_test)

In [13]:
X.shape

(4601, 57)

In [14]:
y.shape

(4601,)

In [15]:
w=np.zeros(X.shape[1])
b=0

In [16]:
w.shape

(57,)

In [20]:
w,b,_,_ =gradient_descent(X_train_norm, y_train, w, b, compute_cost, compute_gradient, alpha=0.003, num_iters=10000)


Iteration    0: Cost     0.73   
Iteration 1000: Cost     0.35   
Iteration 2000: Cost     0.31   
Iteration 3000: Cost     0.29   
Iteration 4000: Cost     0.28   
Iteration 5000: Cost     0.27   
Iteration 6000: Cost     0.26   
Iteration 7000: Cost     0.26   
Iteration 8000: Cost     0.26   
Iteration 9000: Cost     0.25   
Iteration 9999: Cost     0.25   


In [21]:
def predict(X, w, b):
    
    z_wb = np.dot(X, w) + b
    
    f_wb = 1 / (1 + np.exp(-z_wb))
    
    y_pred = (f_wb >= 0.5).astype(int)
    
    return y_pred

In [24]:
y_pred = predict(X_test_norm,w,b)

In [25]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.91


In [26]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

print('Confusion Matrix:')
print(conf_matrix)

Confusion Matrix:
[[511  20]
 [ 67 323]]
